In [1]:
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Literal

In [2]:
def is_dominated(point: List[float], other_points: List[List[float]], criteria: List[Literal['min', 'max']]) -> bool:
    """
    Check if a given point is dominated by any point in a set of other points.

    Args:
        point: The point to check.
        other_points: The set of other points.
        criteria: A list specifying whether each criterion is to be minimized ('min') or maximized ('max').

    Returns:
        True if the point is dominated, False otherwise.
    """
    for other in other_points:
        if all(
            (c == 'min' and other[i] <= point[i]) or (c == 'max' and other[i] >= point[i])
            for i, c in enumerate(criteria)
        ) and any(
            (c == 'min' and other[i] < point[i]) or (c == 'max' and other[i] > point[i])
            for i, c in enumerate(criteria)
        ):
            return True
    return False

In [3]:
def naive_pareto(points: List[List[float]], criteria: List[Literal['min', 'max']]) -> List[np.ndarray]:
    """
    Compute Pareto fronts using the naive approach.

    Args:
        points: A list of points in the solution space.
        criteria: A list specifying whether each criterion is to be minimized ('min') or maximized ('max').

    Returns:
        A list of Pareto fronts, where each front is a numpy array of points.
    """
    remaining = points.copy()
    fronts = []
    while len(remaining) > 0:
        front = []
        to_remove = []
        for point in remaining:
            if not is_dominated(point, remaining, criteria):
                front.append(point)
                to_remove.append(point)
        fronts.append(np.array(front))
        remaining = [p for p in remaining if not any(np.array_equal(p, t) for t in to_remove)]
    return fronts

In [4]:
def divide_and_conquer(points: List[List[float]], criteria: List[Literal['min', 'max']]) -> List[np.ndarray]:
    """
    Compute Pareto fronts using the divide-and-conquer (Kung) approach.

    Args:
        points: A sorted list of points in the solution space.
        criteria: A list specifying whether each criterion is to be minimized ('min') or maximized ('max').

    Returns:
        A list of Pareto fronts, where each front is a numpy array of points.
    """
    if len(points) <= 1:
        return [np.array(points)]

    mid = len(points) // 2
    left = divide_and_conquer(points[:mid], criteria)
    right = divide_and_conquer(points[mid:], criteria)

    merged_points = np.concatenate(left + right)
    return naive_pareto(merged_points.tolist(), criteria)

In [5]:
def visualize_fronts(fronts: List[np.ndarray], title: str, output_file: str) -> None:
    """
    Visualize the first five Pareto fronts and save the plot to a file.

    Args:
        fronts: A list of Pareto fronts.
        title: The title of the plot.
        output_file: The filename for the output plot image.
    """
    colors = plt.colormaps["tab10"]
    plt.figure(figsize=(10, 6))
    for i, front in enumerate(fronts[:5]):
        plt.scatter(front[:, 0], front[:, 1], label=f'Front {i+1}', color=colors(i))
    plt.xlabel('Criterion 1')
    plt.ylabel('Criterion 2')
    plt.title(title)
    plt.legend()
    plt.savefig(output_file)
    plt.close()

In [6]:
def save_fronts_to_files(fronts: List[np.ndarray], base_filename: str) -> None:
    """
    Save each Pareto front to a separate text file.

    Args:
        fronts: A list of Pareto fronts.
        base_filename: The base name for the output files.
    """
    for i, front in enumerate(fronts):
        filename = f"{base_filename}_front_{i+1}.txt"
        np.savetxt(filename, front, delimiter='\t', fmt='%.6f')

In [7]:
if __name__ == "__main__":
    # Load data from the uploaded file
    filepath = "MO-D3R.txt"
    with open(filepath, 'r') as f:
        data = [list(map(float, line.strip().split('\t'))) for line in f]

    points = np.array(data)

    # Define criteria: ['min', 'max'] means first criterion minimizes, second maximizes
    criteria = ['min', 'max']

    # Naive approach
    naive_fronts = naive_pareto(points.tolist(), criteria)
    print(f"Naive approach found {len(naive_fronts)} fronts.")
    save_fronts_to_files(naive_fronts, "naive_pareto")
    visualize_fronts(naive_fronts, "Naive Pareto Fronts", "naive_pareto_fronts.png")

    # Kung's approach
    sorted_points = sorted(points.tolist(), key=lambda x: x[0])
    kung_fronts = divide_and_conquer(sorted_points, criteria)
    print(f"Kung's approach found {len(kung_fronts)} fronts.")
    save_fronts_to_files(kung_fronts, "kung_pareto")
    visualize_fronts(kung_fronts, "Kung's Pareto Fronts", "kung_pareto_fronts.png")

Naive approach found 31 fronts.
Kung's approach found 31 fronts.
